In [ ]:
# @title 8. Validate Model Quality (Combined Traffic Metrics)

# CELL 1 [TAG: parameters]
# ---------------------------------------------------------
# Default parameters (Airflow will OVERWRITE these)
# ---------------------------------------------------------
RUN_TIMESTAMP = "2025-01-01_00-00-00"  # Injected by Airflow
INPUT_TEST_DATA = "s3://models-quality-eval/2025-01-01_00-00-00/test/grouped_segments.pkl"
INPUT_MODEL_DIR = "s3://models-quality-eval/2025-01-01_00-00-00/models/"
OUTPUT_METRICS_PATH = "s3://models-quality-eval/2025-01-01_00-00-00/metrics/quality_metrics.json"
OUTPUT_PLOT_PATH = "s3://models-quality-eval/2025-01-01_00-00-00/metrics/comparison_plots.png"
VEHICLE_TYPE = 1  # Motorcycle

# Quality Thresholds (Applied to Combined Metrics)
MAX_SPEED_DIFF = 5.0  # km/h
MAX_ACCEL_DIFF = 0.5  # m/s²
MAX_RMSE_THRESHOLD = 0.15  # VSP RMSE threshold

# MinIO Credentials
MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports
import pickle
import json
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import s3fs


In [ ]:
# CELL 3: MinIO Configuration
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)


In [ ]:
# CELL 4: Helper Functions (Physics & Generation)
def calculate_vsp(speed_ms, acc_ms2, veh_type=1):
    """Calculates VSP using MOVESTAR coefficients."""
    if veh_type == 1:  # Motorcycle
        A, B, C, M, f = 0.0251, 0.0, 0.000315, 0.285, 0.285
    else:
        A, B, C, M, f = 0.156461, 0.002002, 0.000493, 1.4788, 1.4788

    vsp = (A * speed_ms) + (B * speed_ms**2) + (C * speed_ms**3) + (M * speed_ms * acc_ms2)
    return vsp / f

def get_distribution(data, bins):
    """Get normalized distribution from data."""
    counts, _ = np.histogram(data, bins=bins)
    total = np.sum(counts)
    return counts / total if total > 0 else np.zeros(len(counts))

def generate_synthetic_sequence(matrix, states, length):
    """Generates a random walk using the trained matrix."""
    if matrix is None or length <= 0:
        return np.array([])

    current_idx = np.random.randint(0, len(matrix))
    path = [current_idx]

    for _ in range(length - 1):
        probs = matrix[current_idx]
        if np.sum(probs) == 0:
            next_idx = current_idx
        else:
            next_idx = np.random.choice(len(probs), p=probs)
        path.append(next_idx)
        current_idx = next_idx

    # Decode state indices to Speed values (Column 0)
    return states[path, 0]

def aggregate_metrics(metrics_dict):
    """
    Aggregate per-traffic metrics into combined overall metrics using weighted averaging.
    
    Args:
        metrics_dict: Dictionary with 'Heavy Traffic' and 'Light Traffic' keys
    
    Returns:
        Dictionary with aggregated metrics weighted by sample size
    """
    heavy = metrics_dict.get("Heavy Traffic", {})
    light = metrics_dict.get("Light Traffic", {})
    
    heavy_size = heavy.get("test_sample_size_sec", 0)
    light_size = light.get("test_sample_size_sec", 0)
    total_size = heavy_size + light_size
    
    if total_size == 0:
        return {}
    
    # Weight by sample size
    heavy_weight = heavy_size / total_size
    light_weight = light_size / total_size
    
    # Aggregate all numeric metrics
    aggregated = {
        "avg_speed_real_kmh": heavy.get("avg_speed_real_kmh", 0) * heavy_weight + light.get("avg_speed_real_kmh", 0) * light_weight,
        "avg_speed_synthetic_kmh": heavy.get("avg_speed_synthetic_kmh", 0) * heavy_weight + light.get("avg_speed_synthetic_kmh", 0) * light_weight,
        "speed_difference_kmh": heavy.get("speed_difference_kmh", 0) * heavy_weight + light.get("speed_difference_kmh", 0) * light_weight,
        "avg_accel_real_ms2": heavy.get("avg_accel_real_ms2", 0) * heavy_weight + light.get("avg_accel_real_ms2", 0) * light_weight,
        "avg_accel_synthetic_ms2": heavy.get("avg_accel_synthetic_ms2", 0) * heavy_weight + light.get("avg_accel_synthetic_ms2", 0) * light_weight,
        "accel_difference_ms2": heavy.get("accel_difference_ms2", 0) * heavy_weight + light.get("accel_difference_ms2", 0) * light_weight,
        "std_speed_real_kmh": heavy.get("std_speed_real_kmh", 0) * heavy_weight + light.get("std_speed_real_kmh", 0) * light_weight,
        "std_speed_synthetic_kmh": heavy.get("std_speed_synthetic_kmh", 0) * heavy_weight + light.get("std_speed_synthetic_kmh", 0) * light_weight,
        "std_accel_real_ms2": heavy.get("std_accel_real_ms2", 0) * heavy_weight + light.get("std_accel_real_ms2", 0) * light_weight,
        "std_accel_synthetic_ms2": heavy.get("std_accel_synthetic_ms2", 0) * heavy_weight + light.get("std_accel_synthetic_ms2", 0) * light_weight,
        "vsp_rmse": heavy.get("vsp_rmse", 0) * heavy_weight + light.get("vsp_rmse", 0) * light_weight,
        "total_sample_size_sec": total_size
    }
    
    return aggregated


In [ ]:
# CELL 5: Load Data & Models
print("Loading artifacts...")
print(f"Run Timestamp: {RUN_TIMESTAMP}")
model_base = INPUT_MODEL_DIR.rstrip("/")
try:
    # 1. Test Data (Grouped Segments)
    with fs.open(INPUT_TEST_DATA, 'rb') as f:
        test_segments = pickle.load(f)

    # 2. Models
    with fs.open(f"{model_base}/transition_matrices.pkl", 'rb') as f:
        trans_matrices = pickle.load(f)
    with fs.open(f"{model_base}/state_definitions.pkl", 'rb') as f:
        state_defs = pickle.load(f)

    print("✅ Loaded test data and models.")
except Exception as e:
    print(f"❌ Error loading artifacts: {e}")
    raise


In [ ]:
# CELL 6: Generate & Compare with Kinematic Metrics
vsp_bins = np.arange(-20, 40, 2)
speed_bins = np.arange(0, 120, 5)  # Speed bins for distribution plots
metrics = {}
traffic_labels = {0: "Heavy Traffic", 1: "Light Traffic"}

# Storage for combined plotting
all_real_speed = []
all_syn_speed = []
all_real_vsp = []
all_syn_vsp = []

fig = plt.figure(figsize=(16, 15))
for group_idx in range(2):
    print(f"\n--- Validating Group {group_idx} ({traffic_labels[group_idx]}) ---")

    # A. Prepare Real Test Data
    real_segments = test_segments[group_idx]
    if not real_segments:
        print("Skipping: No test data found.")
        continue

    real_concat = np.concatenate(real_segments)
    real_speed_kmh = real_concat[:, 0]  # km/h
    real_speed_ms = real_speed_kmh / 3.6  # m/s
    real_acc = real_concat[:, 1]  # m/s²

    # B. Generate Synthetic Data
    target_duration = len(real_speed_kmh)
    syn_speed_kmh = generate_synthetic_sequence(trans_matrices[group_idx], state_defs[group_idx], target_duration)

    if len(syn_speed_kmh) == 0:
        print("Skipping: Model failed to generate data.")
        continue

    syn_speed_ms = syn_speed_kmh / 3.6
    syn_acc = np.gradient(syn_speed_ms)

    # C. Kinematic Metrics
    avg_speed_real = np.mean(real_speed_kmh)
    avg_speed_syn = np.mean(syn_speed_kmh)
    avg_accel_real = np.mean(real_acc)
    avg_accel_syn = np.mean(syn_acc)
    std_speed_real = np.std(real_speed_kmh)
    std_speed_syn = np.std(syn_speed_kmh)
    std_accel_real = np.std(real_acc)
    std_accel_syn = np.std(syn_acc)

    speed_diff = abs(avg_speed_real - avg_speed_syn)
    accel_diff = abs(avg_accel_real - avg_accel_syn)

    # D. Speed Distribution (for plotting only)
    speed_dist_real = get_distribution(real_speed_kmh, speed_bins)
    speed_dist_syn = get_distribution(syn_speed_kmh, speed_bins)

    # E. Calculate VSP for RMSE
    vsp_real = calculate_vsp(real_speed_ms, real_acc, VEHICLE_TYPE)
    vsp_syn = calculate_vsp(syn_speed_ms, syn_acc, VEHICLE_TYPE)

    # F. Compare VSP Distributions
    vsp_dist_real = get_distribution(vsp_real, vsp_bins)
    vsp_dist_syn = get_distribution(vsp_syn, vsp_bins)
    vsp_rmse = np.sqrt(np.mean((vsp_dist_real - vsp_dist_syn)**2))

    # G. Store Metrics
    metrics[traffic_labels[group_idx]] = {
        "avg_speed_real_kmh": float(avg_speed_real),
        "avg_speed_synthetic_kmh": float(avg_speed_syn),
        "speed_difference_kmh": float(speed_diff),
        "avg_accel_real_ms2": float(avg_accel_real),
        "avg_accel_synthetic_ms2": float(avg_accel_syn),
        "accel_difference_ms2": float(accel_diff),
        "std_speed_real_kmh": float(std_speed_real),
        "std_speed_synthetic_kmh": float(std_speed_syn),
        "std_accel_real_ms2": float(std_accel_real),
        "std_accel_synthetic_ms2": float(std_accel_syn),
        "vsp_rmse": float(vsp_rmse),
        "test_sample_size_sec": int(target_duration)
    }

    # H. Store data for combined plot
    all_real_speed.append(real_speed_kmh)
    all_syn_speed.append(syn_speed_kmh)
    all_real_vsp.append(vsp_real)
    all_syn_vsp.append(vsp_syn)

    # I. Print Summary
    print(f"  Avg Speed: Real={avg_speed_real:.2f} km/h, Syn={avg_speed_syn:.2f} km/h, Diff={speed_diff:.2f} km/h")
    print(f"  Avg Accel: Real={avg_accel_real:.3f} m/s², Syn={avg_accel_syn:.3f} m/s², Diff={accel_diff:.3f} m/s²")
    print(f"  VSP RMSE: {vsp_rmse:.4f}")

    # J. Create Plots (3x4 grid: Row 1=Heavy, Row 2=Light, Row 3=Combined)
    base_idx = group_idx * 4
    
    # Plot 1: Speed Distribution
    plt.subplot(3, 4, base_idx + 1)
    plt.bar(speed_bins[:-1], speed_dist_real, width=4, alpha=0.5, label='Real', color='blue')
    plt.plot(speed_bins[:-1], speed_dist_syn, color='red', linewidth=2, label='Synthetic')
    plt.title(f"{traffic_labels[group_idx]}\nSpeed Distribution")
    plt.xlabel("Speed (km/h)")
    plt.ylabel("Probability")
    plt.legend()
    
    # Plot 2: VSP Distribution
    plt.subplot(3, 4, base_idx + 2)
    plt.bar(vsp_bins[:-1], vsp_dist_real, width=1.8, alpha=0.5, label='Real', color='blue')
    plt.plot(vsp_bins[:-1], vsp_dist_syn, color='red', linewidth=2, label='Synthetic')
    plt.title(f"VSP Distribution\nRMSE={vsp_rmse:.3f}")
    plt.xlabel("VSP (kW/ton)")
    plt.ylabel("Probability")
    plt.legend()
    
    # Plot 3: Kinematic Comparison - Speed
    plt.subplot(3, 4, base_idx + 3)
    categories = ['Mean', 'Std Dev']
    real_vals = [avg_speed_real, std_speed_real]
    syn_vals = [avg_speed_syn, std_speed_syn]
    x = np.arange(len(categories))
    width = 0.35
    plt.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    plt.bar(x + width/2, syn_vals, width, label='Synthetic', color='red', alpha=0.7)
    plt.ylabel('Speed (km/h)')
    plt.title(f'Speed Statistics\nΔ={speed_diff:.2f} km/h')
    plt.xticks(x, categories)
    plt.legend()
    
    # Plot 4: Kinematic Comparison - Acceleration
    plt.subplot(3, 4, base_idx + 4)
    real_vals = [avg_accel_real, std_accel_real]
    syn_vals = [avg_accel_syn, std_accel_syn]
    plt.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    plt.bar(x + width/2, syn_vals, width, label='Synthetic', color='red', alpha=0.7)
    plt.ylabel('Acceleration (m/s²)')
    plt.title(f'Acceleration Statistics\nΔ={accel_diff:.3f} m/s²')
    plt.xticks(x, categories)
    plt.legend()

# Row 3: Combined Overall Metrics
print("\n--- Computing Combined Overall Metrics ---")
if len(all_real_speed) == 2:
    # Concatenate all data
    combined_real_speed = np.concatenate(all_real_speed)
    combined_syn_speed = np.concatenate(all_syn_speed)
    combined_real_vsp = np.concatenate(all_real_vsp)
    combined_syn_vsp = np.concatenate(all_syn_vsp)
    
    # Compute distributions
    combined_speed_dist_real = get_distribution(combined_real_speed, speed_bins)
    combined_speed_dist_syn = get_distribution(combined_syn_speed, speed_bins)
    combined_vsp_dist_real = get_distribution(combined_real_vsp, vsp_bins)
    combined_vsp_dist_syn = get_distribution(combined_syn_vsp, vsp_bins)
    combined_vsp_rmse = np.sqrt(np.mean((combined_vsp_dist_real - combined_vsp_dist_syn)**2))
    
    # Compute statistics
    combined_avg_speed_real = np.mean(combined_real_speed)
    combined_avg_speed_syn = np.mean(combined_syn_speed)
    combined_speed_diff = abs(combined_avg_speed_real - combined_avg_speed_syn)
    combined_std_speed_real = np.std(combined_real_speed)
    combined_std_speed_syn = np.std(combined_syn_speed)
    
    # Compute acceleration from speed
    combined_real_speed_ms = combined_real_speed / 3.6
    combined_syn_speed_ms = combined_syn_speed / 3.6
    combined_real_accel = np.gradient(combined_real_speed_ms)
    combined_syn_accel = np.gradient(combined_syn_speed_ms)
    combined_avg_accel_real = np.mean(combined_real_accel)
    combined_avg_accel_syn = np.mean(combined_syn_accel)
    combined_accel_diff = abs(combined_avg_accel_real - combined_avg_accel_syn)
    combined_std_accel_real = np.std(combined_real_accel)
    combined_std_accel_syn = np.std(combined_syn_accel)
    
    print(f"  Combined Avg Speed: Real={combined_avg_speed_real:.2f} km/h, Syn={combined_avg_speed_syn:.2f} km/h")
    print(f"  Combined Avg Accel: Real={combined_avg_accel_real:.3f} m/s², Syn={combined_avg_accel_syn:.3f} m/s²")
    print(f"  Combined VSP RMSE: {combined_vsp_rmse:.4f}")
    
    # Plot 1: Combined Speed Distribution
    plt.subplot(3, 4, 9)
    plt.bar(speed_bins[:-1], combined_speed_dist_real, width=4, alpha=0.5, label='Real', color='blue')
    plt.plot(speed_bins[:-1], combined_speed_dist_syn, color='red', linewidth=2, label='Synthetic')
    plt.title(f"Combined Overall\nSpeed Distribution")
    plt.xlabel("Speed (km/h)")
    plt.ylabel("Probability")
    plt.legend()
    
    # Plot 2: Combined VSP Distribution
    plt.subplot(3, 4, 10)
    plt.bar(vsp_bins[:-1], combined_vsp_dist_real, width=1.8, alpha=0.5, label='Real', color='blue')
    plt.plot(vsp_bins[:-1], combined_vsp_dist_syn, color='red', linewidth=2, label='Synthetic')
    plt.title(f"VSP Distribution\nRMSE={combined_vsp_rmse:.3f}")
    plt.xlabel("VSP (kW/ton)")
    plt.ylabel("Probability")
    plt.legend()
    
    # Plot 3: Combined Speed Statistics
    plt.subplot(3, 4, 11)
    categories = ['Mean', 'Std Dev']
    real_vals = [combined_avg_speed_real, combined_std_speed_real]
    syn_vals = [combined_avg_speed_syn, combined_std_speed_syn]
    x = np.arange(len(categories))
    width = 0.35
    plt.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    plt.bar(x + width/2, syn_vals, width, label='Synthetic', color='red', alpha=0.7)
    plt.ylabel('Speed (km/h)')
    plt.title(f'Speed Statistics\nΔ={combined_speed_diff:.2f} km/h')
    plt.xticks(x, categories)
    plt.legend()
    
    # Plot 4: Combined Acceleration Statistics
    plt.subplot(3, 4, 12)
    real_vals = [combined_avg_accel_real, combined_std_accel_real]
    syn_vals = [combined_avg_accel_syn, combined_std_accel_syn]
    plt.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    plt.bar(x + width/2, syn_vals, width, label='Synthetic', color='red', alpha=0.7)
    plt.ylabel('Acceleration (m/s²)')
    plt.title(f'Acceleration Statistics\nΔ={combined_accel_diff:.3f} m/s²')
    plt.xticks(x, categories)
    plt.legend()

plt.tight_layout()

# Save Plot immediately after creating it (before moving to next cell)
img_buf = io.BytesIO()
fig.savefig(img_buf, format='png', dpi=150)
img_buf.seek(0)
with fs.open(OUTPUT_PLOT_PATH, 'wb') as f:
    f.write(img_buf.getbuffer())
print(f"\n✅ Plots saved to {OUTPUT_PLOT_PATH}")
plt.close(fig)


In [ ]:
# CELL 7: Aggregate Metrics & Evaluate Pass/Fail
print("\n=== PER-TRAFFIC METRICS ===")
print(json.dumps(metrics, indent=2))

# Aggregate metrics across traffic conditions
overall_metrics = aggregate_metrics(metrics)

print("\n=== COMBINED OVERALL METRICS ===")
print(json.dumps(overall_metrics, indent=2))

# Determine pass/fail status based on OVERALL metrics
failures = []
if overall_metrics.get('speed_difference_kmh', 0) > MAX_SPEED_DIFF:
    failures.append(f"Speed diff {overall_metrics['speed_difference_kmh']:.2f} > {MAX_SPEED_DIFF} km/h")
if overall_metrics.get('accel_difference_ms2', 0) > MAX_ACCEL_DIFF:
    failures.append(f"Accel diff {overall_metrics['accel_difference_ms2']:.3f} > {MAX_ACCEL_DIFF} m/s²")
if overall_metrics.get('vsp_rmse', 0) > MAX_RMSE_THRESHOLD:
    failures.append(f"VSP RMSE {overall_metrics['vsp_rmse']:.4f} > {MAX_RMSE_THRESHOLD}")

# Add validation status to overall metrics
overall_metrics['validation_status'] = 'FAILED' if failures else 'PASSED'
overall_metrics['failures'] = failures

# Save Metrics JSON with new structure
final_metrics = {
    "overall": overall_metrics,
    "heavy_traffic": metrics.get("Heavy Traffic", {}),
    "light_traffic": metrics.get("Light Traffic", {})
}
with fs.open(OUTPUT_METRICS_PATH, 'w') as f:
    json.dump(final_metrics, f, indent=2)
print(f"✅ Metrics saved to {OUTPUT_METRICS_PATH}")

# Quality Gate Summary
print(f"\n=== VALIDATION SUMMARY ===")
print(f"Status: {overall_metrics['validation_status']}")
print(f"Quality Thresholds:")
print(f"  - Max Speed Difference: {MAX_SPEED_DIFF} km/h")
print(f"  - Max Acceleration Difference: {MAX_ACCEL_DIFF} m/s²")
print(f"  - Max VSP RMSE: {MAX_RMSE_THRESHOLD}")

if failures:
    print("\n⚠️  Model Quality Validation Failed (Results saved for analysis):")
    for failure in failures:
        print(f"  - {failure}")
    print("\nNote: This validation compares combined metrics across both heavy and light traffic.")
    print("The model is suitable for comparing different drive cycle generation methods.")
else:
    print("\n✅ Model Quality Validation Passed.")
    print("Combined metrics across traffic scenarios meet all quality thresholds.")
    print("This model can be compared with other drive cycle generation methods.")
